In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
from tensorflow.keras.utils import Sequence, to_categorical, plot_model
from keras.layers import Conv2D, Dropout, MaxPooling2D, UpSampling2D, concatenate, Input
from keras.models import Model
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
df = pd.read_csv('class_dict_seg.csv')
df.head()

In [ ]:
class_list = df.name.values.tolist()
print(class_list)
len(class_list)

In [ ]:
from keras_segmentation.models.unet import vgg_unet

model = vgg_unet(n_classes = len(class_list), input_height = 256, input_width = 256)

In [ ]:
import tensorflow as tf
callback = tf.keras.callbacks.EarlyStopping(monitor = "val_loss", mode = "min", patience = 4)
model.compile(optimizer='adam', loss=['categorical_crossentropy'], metrics=['accuracy'])

h = model.train(train_images = 'image', train_annotations = 'label', 
            val_images = 'val_img', val_annotations = 'val_lab', 
            checkpoints_path = '/tmp/vgg_unet_1', steps_per_epoch = 40, epochs = 50, callbacks = callback)

In [ ]:
model.save('semantic_segmentation_model.h5')

In [ ]:
from keras_segmentation.models.unet import vgg_unet
model = vgg_unet(n_classes=len(class_list), input_height=256, input_width=256)
model.load_weights('semantic_segmentation_model.h5')

In [ ]:
import time
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

start = time.time()

input_image = "test/ta/e19.jpg"
out = model.predict_segmentation(
    inp=input_image,
    out_fname="out.png"
)

fig, axs = plt.subplots(1, 2, figsize=(20, 20), constrained_layout=True)

img_orig = Image.open(input_image)
axs[0].imshow(img_orig)
axs[0].set_title('C17.jpg')
axs[0].grid(False)

axs[1].imshow(out)
axs[1].set_title('prediction C17')
axs[1].grid(False)





done = time.time()
elapsed = done - start